In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
from torchvision import datasets, transforms, models
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [2]:
# input dataset.
data_dir = '../input/face-mask-12k-images-dataset/Face Mask Dataset'

train_transforms = transforms.Compose([transforms.RandomRotation(5),
                                      transforms.RandomResizedCrop(224),
                                      transforms.ToTensor()])
test_transforms = transforms.Compose([transforms.RandomRotation(5),
                                     transforms.RandomResizedCrop(224),
                                     transforms.ToTensor()])

train_data = datasets.ImageFolder(data_dir + '/Train', transform=train_transforms)
test_data = datasets.ImageFolder(data_dir + '/Test', transform=test_transforms)

trainloader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle = True)

In [3]:
#Resnet18 model finetuning
model = models.resnet18(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

num_ftrs = model.fc.in_features

model.fc = nn.Linear(num_ftrs, 2)

input_size = 224

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

params_to_update = model.parameters()

optimizer = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

criterion = nn.CrossEntropyLoss()

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [4]:
#calculating train and test accuracy.
def train_accuracy(net, trainset_loader, epoch):

    net.eval()
    correct = 0
    total = 0
    for data in trainset_loader:
        images, labels = data
        images, labels = Variable(images).cuda(), labels.cuda()
        output = net(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
    print('Training Accuracy of the network after epoch '+str(epoch+1)+' is: ' + str(accuracy))
    return accuracy

def test_accuracy(net, testset_loader, epoch):
    # Test the model
    net.eval()
    correct = 0
    total = 0
    for data in testset_loader:
        images, labels = data
        images, labels = Variable(images).cuda(), labels.cuda()
        output = net(images)
        _, predicted = torch.max(output.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        accuracy = 100 * correct / total
    print('Testing Accuracy of the network after epoch '+str(epoch+1)+' is: ' + str(accuracy))
    return accuracy

In [5]:
#Training
loss_l = []
test_l = []
train_l = []

for epoch in range(18):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader):
        input_data, labels = data # data is a list of 2, the first element is 4*3*32*32 (4 images) the second element is a list of 4 (classes)
        input_data, labels = Variable(input_data).cuda(),Variable(labels).cuda()
        optimizer.zero_grad() # every time reset the parameter gradients to zero
        # forward backward optimize
        output = model(input_data)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # print the loss after every epoch
    t = len(trainloader.dataset)
    print('loss in epoch ' + str(epoch + 1) + ': ' + str(running_loss / t))
    loss_l.append(running_loss / t)
    
    if (epoch + 1)%3 == 0:
        Test_acc = test_accuracy(model, testloader, epoch)
        train_acc = train_accuracy(model, trainloader, epoch)
        test_l.append(Test_acc)
        train_l.append(train_acc)
        
print('Finished Training')

RuntimeError: Tensor for 'out' is on CPU, Tensor for argument #1 'self' is on CPU, but expected them to be on GPU (while checking arguments for addmm)

In [ ]:
epochs = [3,6,9,12,15,18]
resnet18_loss = loss_l
resnet18_train = train_l
resnet18_test = test_l
plt.plot(range(18),resnet18_loss,'-.')
plt.title('Cross entropy loss curve')
plt.xlabel('No. of Epochs')
plt.ylabel('Cross Entropy loss')
plt.figure()
plt.plot(epochs,resnet18_train,'-.')
plt.plot(epochs,resnet18_test,'-.')
plt.xlabel('No. of Epochs')
plt.ylabel('Accuracy')
plt.legend(['Training accuracy','Testing accuracy'])

In [ ]:
#alexnet and logistic regression

model_ft = models.alexnet(pretrained=True)

#extracting train features and labels.
features = None
labels = None

for i, data in enumerate(trainloader):
    if(i%3 == 0):
        input_data, y = data
        output = model_ft.features(input_data)
        output = output.view(-1, 6*6*256)
        if features is not None:
            features = torch.cat((features, output), 0)
            labels = torch.cat((labels, y), 0)
        else:
            features = output
            labels = y
            
feat_df = pd.DataFrame(features.detach().numpy(), columns=[f'img_feature_{n}' for n in range(features.size(-1))])
labels = pd.DataFrame(labels.detach().numpy())

In [ ]:
#extracting test features and labels.
test_features = None
test_labels = None
for i, data in enumerate(testloader):
    input_data, y = data
    output = model_ft.features(input_data)
    output = output.view(-1, 6*6*256)
    if test_features is not None:
        test_features = torch.cat((test_features, output), 0)
        test_labels = torch.cat((test_labels, y), 0)
    else:
        test_features = output
        test_labels = y

test_df = pd.DataFrame(test_features.detach().numpy(), columns=[f'img_feature_{n}' for n in range(test_features.size(-1))])
test_labels = pd.DataFrame(test_labels.detach().numpy())

In [ ]:
#training logistic regression model.
logreg = LogisticRegression()
logreg.fit(feat_df,labels)

#predicting on test data.
predictions = logreg.predict(test_df)

#printing the accuracy of the model on the test data.
print(classification_report(test_labels,predictions))

print(accuracy_score(test_labels, predictions))

In [ ]:
import pickle
filename = 'finalized_model.sav'
pickle.dump(logreg, open(filename, 'wb'))